<a href="https://colab.research.google.com/github/jsebastianquiroga/Topicos_analitica/blob/main/Notebook/Empirical_network_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img style="float: center; padding-right: 10px;" src="https://www.pikpng.com/pngl/b/467-4670553_universidad-javeriana-esslingen-am-neckar-clipart.png" width="100" height="150">



<h1> <strong>Tópicos avanzados en analítica.</strong></h1> 
<h2> Para: Luis Gabriel Moreno Sandoval.</br></h2>
<h2>Integrantes:</br></h2>
<h2>Alejandro Vivas.</br></h2>
<h2>Johan Sebastian Muñoz.</br></h2>
<h2>Juan Sebastián Quiroga Bernal. </h2>

<h1> <strong>Workshop 1: Empirical Network  - Twitter.</strong></h1> 

En el presnte Notebook, se desarrollar una funcion que permita extraer los tuits que se desen con la API de snscrape.


Este docuemnto esta dividio en las siguientes partes:
<ol>
<li> Funcion - Extraer Tuits.. </li>
<li> Creación de Red de los Hastags. </li>
<li> Creación de Red que asocie las palabras mas usadas. </li>
<li> Conclusiones.</li>
</ol>

***

In [1]:
!pip install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 KB 2.0 MB/s eta 0:00:00


In [2]:
#creación de funcion.

#import the libraries to be use.
import snscrape.modules.twitter as sntwitter
import pandas as pd

query = "ptyhon"
tweets = []
limits = 100

for tweet in sntwitter.TwitterSearchScraper(query).get_items():

  if len(tweets) == limits:
    break
  else:
    tweets.append([tweet.date, tweet.user.username, tweet.content])

df =pd.DataFrame(tweets, columns =['Date', 'User', 'Tweet'])

df

  # print(vars(tweet))


<ipython-input-2-dd5c721e3a40>:16: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  tweets.append([tweet.date, tweet.user.username, tweet.content])


,Date,User,Tweet
0,2023-03-18 12:59:23+00:00,Ptyhon1,@BAlpargu @karargah_haber Tayyip gitsin diye s...
1,2023-03-18 12:56:59+00:00,Ptyhon1,@solcugazete Mankeni sikemediği için sinirlene...
2,2023-03-18 12:56:06+00:00,Ptyhon1,@jahreindota Ne demeh yeri yoh yav cennet anal...
3,2023-03-18 12:53:39+00:00,Ptyhon1,@solcugazete Allah afk
4,2023-03-18 06:13:15+00:00,Ptyhon1,@routesosyal1 @muhendisIik_ Cevap 18.
...,...,...,...
95,2023-02-11 16:54:58+00:00,yurideveloper,Todo desenvolvedor precisa de uma linguagem de...
96,2023-02-09 12:02:13+00:00,juliancamposes,¿Quieres una aplicación más al detalle de por ...
97,2023-02-06 20:29:32+00:00,rekman56,"for i in range(1,24):\ntsais le mec qui confon..."
98,2023-01-28 23:27:13+00:00,roolbackgramer,動的言語 Rubyをやっても日本全国には仕事がない。せいぜい東京のWeb限定の中小企業のみだ...


##2

In [44]:
import snscrape.modules.twitter as sntwitter
import networkx as nx
import plotly.graph_objects as go
import re
import pandas as pd

import snscrape.modules.twitter as sntwitter
import networkx as nx
import plotly.graph_objects as go
import re
import pandas as pd
import plotly.express as px
import datetime as dt
import pytz

class TwitterGraph:
    def __init__(self, query, search_method, max_tweets=100, start_date=None, end_date=None):
        self.query = query
        self.search_method = search_method
        self.max_tweets = max_tweets
        self.start_date = start_date
        self.end_date = end_date
        self.graph = nx.Graph()
        self.data = None

    def _scrape_tweets(self):
        # construct search query based on search method and time range filters
        if self.start_date and self.end_date:
            date_range_query = f'since:{self.start_date} until:{self.end_date}'
            search_query = f'{self.query} {date_range_query}'
        else:
            search_query = self.query

        # create a scraper object based on the search method
        if self.search_method == 'keyword':
            scraper = sntwitter.TwitterSearchScraper(search_query)
        elif self.search_method == 'hashtag':
            scraper = sntwitter.TwitterSearchScraper(f'#{search_query}')
        elif self.search_method == 'username':
            scraper = sntwitter.TwitterUserScraper(search_query)
        elif self.search_method == 'list':
            scraper = sntwitter.TwitterListScraper(search_query)
        elif self.search_method == 'advanced':
            scraper = sntwitter.TwitterSearchScraper(search_query)
        elif self.search_method == 'location':
            scraper = sntwitter.TwitterSearchScraper(search_query)
        else:
            raise ValueError("Invalid search_method. Must be 'keyword', 'hashtag', 'username', 'list', 'advanced', or 'location'.")

        tweets = []
        count = 0
        # fetch tweets until we reach the desired number of tweets
        for tweet in scraper.get_items():
            if count >= self.max_tweets:
                break
            tweets.append(tweet)
            count += 1

        # filter tweets by date range if start_date and end_date are specified
        if self.start_date and self.end_date:
            start_date_dt = dt.datetime.strptime(self.start_date, '%Y-%m-%d').replace(tzinfo=pytz.UTC)
            end_date_dt = dt.datetime.strptime(self.end_date, '%Y-%m-%d').replace(tzinfo=pytz.UTC)
            filtered_tweets = []
            for tweet in tweets:
                if tweet.date >= start_date_dt and tweet.date <= end_date_dt:
                    filtered_tweets.append(tweet)
            tweets = filtered_tweets

        # extract tweet information
        data = []
        for tweet in tweets:
            # extract hashtags
            hashtags = re.findall(r'#\w+', tweet.rawContent)
            for i, h1 in enumerate(hashtags):
                for h2 in hashtags[i+1:]:
                    # create an edge between users who have used the same hashtag
                    self.graph.add_edge(tweet.user.username, h1)
                    self.graph.add_edge(tweet.user.username, h2)
            
            data.append({
                'tweet_id': tweet.id,
                'username': tweet.user.username,
                'Replay_to':tweet.inReplyToUser,
                'user_mentioned': tweet.mentionedUsers,
                'retweet' : tweet.retweetedTweet,
                'quoted_tweet' : tweet.quotedTweet,
                'content': tweet.rawContent,
                'date': tweet.date,
                'retweets': tweet.retweetCount,
                'likes': tweet.likeCount,
                'quotes': tweet.quoteCount,
                'replies': tweet.replyCount,
                'url': tweet.url,
                'place': tweet.place,
                'hashtags': tweet.hashtags,
                'cashtags' : tweet.cashtags
            })

        # Create a DataFrame from the extracted data
        self.data = pd.DataFrame(data)
        return data

    def build_graph(self):
        self._scrape_tweets()
        self.graph.remove_nodes_from(list(nx.isolates(self.graph)))

        # compute node positions for the graph
        node_positions = nx.spring_layout(self.graph)

        # create a dictionary that maps usernames to node positions
        user_positions = {}
        for node, pos in node_positions.items():
            user_positions[node] = pos
            self.graph.nodes[node]['pos'] = pos

        return user_positions
        
    def visualize_graph(self):
        # get the node positions from the built graph
        node_positions = self.build_graph()

        # create a dataframe from the node positions
        pos_df = pd.DataFrame(node_positions, index=['x', 'y']).T
        pos_df.index.name = 'node'

        # add the node positions to the graph
        nx.set_node_attributes(self.graph, pos_df.to_dict('index'))

        # create a plotly figure object
        fig = px.scatter(pos_df, x='x', y='y', text=pos_df.index, custom_data=[pos_df.index])

        # add edges to the figure
        for edge in self.graph.edges:
            x0, y0 = self.graph.nodes[edge[0]]['x'], self.graph.nodes[edge[0]]['y']
            x1, y1 = self.graph.nodes[edge[1]]['x'], self.graph.nodes[edge[1]]['y']
            fig.add_trace(
                go.Scatter(x=[x0, x1], y=[y0, y1], mode='lines', line=dict(color='gray'))
            )

        # configure the figure layout
        fig.update_traces(textposition='top center', marker=dict(size=10, color='lightblue'), hovertemplate='Username: %{customdata[0]}<extra></extra>')
        fig.update_layout(title=f'Twitter Graph for {self.query}', title_font_size=30,
                          xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                          yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                          hovermode='closest')

        # display the figure
        fig.show()

In [73]:
# Instantiate the TwitterGraph class with the desired parameters
tg = TwitterGraph(query='Chelsea', search_method='hashtag', max_tweets=10, start_date='2023-03-15', end_date='2023-03-18')
tweet = tg._scrape_tweets()
#df = pd.DataFrame(tweet)
#df.user_mentioned = df.user_mentioned.iloc[:][0].username
#df
tweet



[{'tweet_id': 1636878854638673923,
  'username': 'transfer_review',
  'Replay_to': None,
  'user_mentioned': [User(username='MattHughesDM', id=96757955, displayname='Matt Hughes', rawDescription=None, renderedDescription=None, descriptionLinks=None, verified=None, created=None, followersCount=None, friendsCount=None, statusesCount=None, favouritesCount=None, listedCount=None, mediaCount=None, location=None, protected=None, link=None, profileImageUrl=None, profileBannerUrl=None, label=None)],
  'retweet': None,
  'quoted_tweet': None,
  'content': '#MasonMount has appointed a new agent, Neil Fewings from Wasserman Media Group, to manage a potential transfer from #Chelsea — the strongest indication yet that he is set to leave #StamfordBridge at the end of the season.\n\n(@MattHughesDM)\n\n#PL #Transfers\nhttps://t.co/sRUGtnznxd',
  'date': datetime.datetime(2023, 3, 17, 23, 55, 5, tzinfo=datetime.timezone.utc),
  'retweets': 0,
  'likes': 0,
  'quotes': 0,
  'replies': 1,
  'url': 'https

In [72]:
tweet.mentionedUsers[0]

AttributeError: ignored

In [66]:
type(tweet)

list

In [59]:
df.user_mentioned

0                   [https://twitter.com/MattHughesDM]
1                [https://twitter.com/TuttoMercatoWeb]
2                                                 None
3                  [https://twitter.com/BabilonBetcom]
4                                                 None
5                [https://twitter.com/ThibaudVezirian]
6                                                 None
7    [https://twitter.com/MattHughesDM, https://twi...
8                                                 None
9                    [https://twitter.com/Matt_Law_DT]
Name: user_mentioned, dtype: object

In [18]:
re.findall(r'#[^\s#]\S*', df.content[1])

['#Chelsea',
 '#Mendy',
 '#Onana',
 '#Inter',
 '#Calciomercato',
 '#Transfers',
 '#PremierLeague',
 '#StayTuned']

In [13]:
tg.build_graph()

# Build and visualize the graph
tg.visualize_graph()


In [23]:
# Instantiate the TwitterGraph class with the desired parameters
tg_1 = TwitterGraph(query='petrogustavo', search_method='username', max_tweets=20)#, start_date='2023-03-15', end_date='2023-03-18')
tweet_1 = tg_1._scrape_tweets()
df_1 = pd.DataFrame(tweet_1)
df_1

,tweet_id,username,content,date,retweets,likes,quotes,replies,url
0,1637097300244717573,petrogustavo,#ColombiaVaBien,2023-03-18 14:23:06+00:00,90,250,5,63,https://twitter.com/petrogustavo/status/163709...
1,1637095127721058311,petrogustavo,5.000 mujeres saldrán de la cárcel 5 días a la...,2023-03-18 14:14:28+00:00,182,499,53,288,https://twitter.com/petrogustavo/status/163709...
2,1637093132180377606,petrogustavo,La rebaja de tasas de interés del Banco Agrari...,2023-03-18 14:06:33+00:00,200,488,11,59,https://twitter.com/petrogustavo/status/163709...
3,1637092121311162373,petrogustavo,Entre la Fuerza Aérea y la Armada se intercept...,2023-03-18 14:02:32+00:00,372,787,27,167,https://twitter.com/petrogustavo/status/163709...
4,1637089681740296193,petrogustavo,No hay un solo medio en la producción que no s...,2023-03-18 13:52:50+00:00,204,461,8,106,https://twitter.com/petrogustavo/status/163708...
5,1637082089022521344,petrogustavo,La revitalización de la Selva Amazónica y la r...,2023-03-18 13:22:40+00:00,352,823,4,81,https://twitter.com/petrogustavo/status/163708...
6,1636947440015163392,petrogustavo,Nuevo decomiso de cocaína en el golfo de Urabá...,2023-03-18 04:27:37+00:00,2016,4302,110,683,https://twitter.com/petrogustavo/status/163694...
7,1636928065044348931,petrogustavo,Titular 13.000 hectáreas de la reserva indígen...,2023-03-18 03:10:38+00:00,352,865,5,37,https://twitter.com/petrogustavo/status/163692...
8,1636928062351720449,petrogustavo,"En Quinchía, Risaralda en "" El gobierno escuch...",2023-03-18 03:10:37+00:00,1215,2774,43,288,https://twitter.com/petrogustavo/status/163692...
9,1636871943570817025,petrogustavo,"En Quinchía, Risaralda en ""La presidencia escu...",2023-03-17 23:27:37+00:00,704,1825,15,198,https://twitter.com/petrogustavo/status/163687...
